# Census Data

This notebook imports 4 csv files that contain census data for the 207 tracts in Suffolk County, MA. The data has been normalized (credit to Brown University) to reflect the tracts from the 2010 census. After a number of pre-processing steps, I calculate the percentage change between the two censuses with an end goal of using k-means clustering on the tracts.

In [1]:
import pandas as pd

In [203]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

## Data pre-processing (2000 census, full count)

In [408]:
df_2000 = pd.read_csv('LTDB_Std_2000_fullcount.csv',sep=',', engine='python')

In [409]:
df_MA_full_2000 = df_2000[(df_2000['state']=='MA') & (df_2000['county']=='Suffolk County')]

In [410]:
df_MA_full_2000.drop(columns=['placefp10','cbsa10','metdiv10','ccflag10'],axis=1,inplace=True)

In [411]:
df_MA_full_2000.reset_index(inplace=True, drop=True)

In [412]:
# rename tract id column to match 2000
df_MA_full_2000.rename(columns={'TRTID10':'tractid'},inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [413]:
# make columns in same order as 2010 so we can iterate over the df
df_MA_full_2000 = df_MA_full_2000[['tractid', 'state', 'county', 'tract', 'POP00', 'NHWHT00', 'NHBLK00',
       'NTV00', 'ASIAN00', 'HISP00', 'HAW00', 'INDIA00', 'CHINA00', 'FILIP00',
       'JAPAN00', 'KOREA00', 'VIET00', 'MEX00', 'PR00', 'CUBAN00', 'FAMILY00','FHH00','HU00',
       'VAC00', 'OHU00','OWN00', 'RENT00', 'A18UND00', 'A60UP00', 'A75UP00', 'AGEWHT00',
       'A15WHT00', 'A60WHT00', 'AGEBLK00', 'A15BLK00', 'A60BLK00', 'AGEHSP00',
       'A15HSP00', 'A60HSP00', 'AGEASN00','A15ASN00','A60ASN00','AGENTV00','A15NTV00', 'A60NTV00',
       'GlobD00', 'GlobG00']]

In [414]:
df_MA_full_2000[df_MA_full_2000['tractid']==25025981800]

,tractid,state,county,tract,POP00,NHWHT00,NHBLK00,NTV00,ASIAN00,HISP00,HAW00,INDIA00,CHINA00,FILIP00,JAPAN00,KOREA00,VIET00,MEX00,PR00,CUBAN00,FAMILY00,FHH00,HU00,VAC00,OHU00,OWN00,RENT00,A18UND00,A60UP00,A75UP00,AGEWHT00,A15WHT00,A60WHT00,AGEBLK00,A15BLK00,A60BLK00,AGEHSP00,A15HSP00,A60HSP00,AGEASN00,A15ASN00,A60ASN00,AGENTV00,A15NTV00,A60NTV00,GlobD00,GlobG00
202,25025981800,MA,Suffolk County,Census Tract 9818,5.011266,4.136201,0.239674,0.013373,0.308941,0.276137,0.000366,0.059378,0.119262,0.016541,0.049075,0.034982,0.009474,0.041963,0.071512,0.029322,1.046647,0.041287,2.364758,0.062389,2.302369,1.378303,0.924067,0.65182,0.987342,0.423244,4.136201,0.452929,0.916435,0.213941,0.038331,0.023326,0.276137,0.050268,0.021383,0.285644,0.028801,0.016513,0.004265,0.001126,0.0,wba,Semi global


## Data pre-processing (2010 census, full count)

In [415]:
df_2010 = pd.read_csv('LTDB_Std_2010_fullcount.csv',sep=',', engine='python')

In [416]:
df_MA_full_2010 = df_2010[(df_2010['state']=='MA') & (df_2010['county']=='Suffolk County')]

In [417]:
df_MA_full_2010.reset_index(inplace=True,drop=True)

## Calculate change b/w censuses (full count)

In [166]:
## New df for change in values between 2000 and 2010

In [418]:
df_joined = df_MA_full_2000.merge(df_MA_full_2010, on='tractid')

In [419]:
df_joined.drop(columns=['state_y','county_y','tract_y'],inplace=True)

In [420]:
df_joined.drop(columns=['GlobD00','GlobG00','GlobD10','GlobG10'],axis=1,inplace=True)

In [429]:
# for 
i = 4
j = 45
# stop once you get to final column
while j < 86:
    df_joined[f'{df_joined.columns[i]}_POC'] = df_joined.iloc[:,4:].apply(lambda x: 100*
                                    ((x[df_joined.columns[j]]-x[df_joined.columns[i]])
                                     /x[df_joined.columns[i]]),axis=1)
    i +=1
    j +=1

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [227]:
df_joined.to_csv('Full_count_census_changes.csv')

## Data pre-processing (2000 Census, Sample count)

In [484]:
df_2000_2 = pd.read_csv('LTDB_Std_2000_Sample.csv',sep=',', engine='python')

In [485]:
df_MA_sample_2000 = df_2000_2[(df_2000_2['state']=='MA') & (df_2000_2['county']=='Suffolk County')]

In [486]:
df_MA_sample_2000.drop(columns=['placefp10','cbsa10','metdiv10','ccflag10'],axis=1,inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [487]:
df_MA_sample_2000.drop(columns=['state','county','tract'],axis=1,inplace=True)

In [488]:
# this column exists in full count df
df_MA_sample_2000.drop(columns=['POP00SF3'],axis=1,inplace=True)

In [489]:
# drop columns that are not variables in 2010
df_MA_sample_2000.drop(columns=['DIS00','CNI16U00'],axis=1,inplace=True)

In [ ]:
## convert dtypes to floats

In [510]:
# fill in empty strings with 0's
df_MA_sample_2000['HINC00'] = pd.to_numeric(df_MA_sample_2000['HINC00'],errors='coerce')
df_MA_sample_2000['HINCW00'] = pd.to_numeric(df_MA_sample_2000['HINC00'],errors='coerce')
df_MA_sample_2000['HINCB00'] = pd.to_numeric(df_MA_sample_2000['HINCB00'],errors='coerce')
df_MA_sample_2000['HINCH00'] = pd.to_numeric(df_MA_sample_2000['HINCH00'],errors='coerce')
df_MA_sample_2000['HINCA00'] = pd.to_numeric(df_MA_sample_2000['HINCA00'],errors='coerce')
df_MA_sample_2000['MHMVAL00'] = pd.to_numeric(df_MA_sample_2000['MHMVAL00'],errors='coerce')
df_MA_sample_2000['MRENT00'] = pd.to_numeric(df_MA_sample_2000['MRENT00'],errors='coerce')

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [519]:
df_MA_sample_2000.rename(columns={'TRTID10':'tractid'},inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


## Data pre-processing (2010 Census, Sample count)

In [381]:
df_2010_2 = pd.read_csv('LTDB_Std_2010_Sample.csv',sep=',', engine='python')
df_MA_sample_2010 = df_2010_2[(df_2010_2['statea']==25) & (df_2010_2['countya']==25)]

In [ ]:
# most of the variables in the sample files are derived from values in the full count files, so we are going
# to ignore those
# additionally, 2010 ACS has more variables - we are selecting only those that are in 2000 also

In [382]:
df_MA_sample_2010.head()

,tractid,statea,countya,tracta,pnhwht12,pnhblk12,phisp12,pntv12,pasian12,phaw12,pindia12,pchina12,pfilip12,pjapan12,pkorea12,pviet12,p15wht12,p65wht12,p15blk12,p65blk12,p15hsp12,p65hsp12,p15ntv12,p65ntv12,p15asn12,p65asn12,pmex12,pcuban12,ppr12,pruanc12,pitanc12,pgeanc12,piranc12,pscanc12,pfb12,pnat12,p10imm12,prufb12,pitfb12,pgefb12,pirfb12,pscfb12,polang12,plep12,phs12,pcol12,punemp12,pflabf12,pprof12,pmanuf12,psemp12,pvet12,p65pov12,ppov12,pwpov12,pnapov12,pfmpov12,pbpov12,phpov12,papov12,pvac12,pown12,pmulti12,p30old12,p18und12,p60up12,p75up12,pmar12,pwds12,pfhh12,p10yrs12,ageblk12,agentv12,agewht12,agehsp12,india12,filip12,japan12,korea12,viet12,pop12,nhwht12,nhblk12,ntv12,hisp12,asian12,haw12,china12,a15wht12,a65wht12,a15blk12,a65blk12,a15hsp12,a65hsp12,a15ntv12,a65ntv12,ageasn12,a15asn12,a65asn12,mex12,pr12,cuban12,geanc12,iranc12,itanc12,ruanc12,fb12,nat12,itfb12,rufb12,ag5up12,irfb12,gefb12,scanc12,n10imm12,olang12,lep12,scfb12,ag25up12,dfmpov12,hh12,hinc12,hincb12,hincw12,hinch12,incpc12,ag18cv12,vet12,empclf12,dpov12,npov12,dbpov12,nbpov12,dnapov12,nnapov12,dwpov12,nwpov12,dhpov12,nhpov12,hhb12,hhw12,hhh12,hs12,col12,clf12,unemp12,dflabf12,flabf12,prof12,manuf12,semp12,hha12,hinca12,n65pov12,nfmpov12,napov12,dapov12,family12,hu12,vac12,ohu12,own12,rent12,dmulti12,mrent12,mhmval12,multi12,h30old12,h10yrs12,a18und12,a60up12,a75up12,ag15up12,12-Mar,wds12,fhh12
32317,25025000100,25,25,100,67.260002,3.80,17.580000,0.0,9.790000,0.0,1.69,6.35,0.00,0.64,0.33,0.00,5.32,16.030001,25.770000,0.000000,20.660000,0.00,-999.0,-999.0,9.35,0.00,0.25,0.67,5.24,1.69,10.65,3.05,13.060000,2.99,29.030001,15.30,9.48,0.80,0.50,0.00,0.42,0.39,38.450001,15.54,54.939999,30.780001,6.06,62.259998,22.500000,7.83,7.13,5.51,1.37,14.660000,18.549999,-999.0,3.41,6.380000,2.070000,14.730000,10.17,24.570000,78.639999,94.529999,10.81,16.219999,5.77,27.690001,17.10,10.240000,76.889999,163,0,2426,634,61,0,23,12,0,3607,2426,137,0,634,353,0,229,129,389,42,0,131,0,0,0,353,33,0,9,189,24,110,471,384,61,1047,552,18,29,3358,15,0,108,342,1291,522,14,2563,674,1510,53571,31759,54777,46106,27564,3192,176,2133,3574,524,141,9,0,0,2415,448,629,13,38,1184,143,1408,789,2260,137,1587,988,480,167,152,136,62115.0,49,23,52,353,674,1681,171,1510,371,1139,1681,1348,348200,1322,1589,1161,390,585,208,3258,902,557,69
32318,25025000201,25,25,201,67.959999,3.85,16.879999,0.0,9.240000,0.0,0.76,6.56,1.02,0.89,0.00,0.00,4.64,7.970000,0.000000,16.530001,13.210000,14.72,-999.0,-999.0,0.00,10.69,1.02,3.60,5.99,0.32,10.41,0.64,24.650000,1.11,25.129999,14.55,7.07,0.00,1.18,0.86,2.10,0.00,29.030001,6.06,32.290001,53.320000,1.89,79.230003,45.750000,5.98,8.02,3.54,0.64,10.130000,6.420000,-999.0,5.09,49.590000,10.380000,6.550000,13.34,29.379999,83.279999,95.980003,6.37,14.780000,5.35,27.059999,9.05,10.180000,70.400002,121,0,2134,530,24,32,28,0,0,3140,2134,121,0,530,290,0,206,99,170,0,20,70,78,0,0,290,0,31,32,188,113,20,774,327,10,789,457,37,0,3038,66,27,35,222,882,184,0,2273,570,1358,65506,9500,76218,65000,38128,2940,104,2343,3140,318,121,60,0,0,2134,137,530,55,52,996,184,734,1212,2381,45,1671,1324,1072,140,188,73,58393.0,20,29,19,290,570,1567,209,1358,399,959,1567,1405,532000,1305,1504,956,200,464,168,2971,804,269,58
32319,25025000202,25,25,202,53.660000,8.54,17.190001,0.0,17.280001,0.0,0.00,9.42,0.35,0.00,0.00,1.91,6.87,10.730000,22.370001,0.000000,17.139999,0.79,-999.0,-999.0,7.74,27.33,1.26,0.00,0.74,2.37,4.26,1.36,24.180000,0.93,36.240002,17.66,12.23,0.49,0.66,0.52,1.42,0.00,41.509998,12.42,32.820000,43.119999,9.60,55.040001,45.950001,1.62,8.85,2.96,0.35,18.200001,13.220000,-999.0,9.33,28.030001,35.869999,12.320000,9.55,32.919998,78.570000,96.260002,13.43,16.180000,7.34,33.090000,13.30,17.870001,62.799999,371,0,1966,630,0,13,0,0,70,3664,1966,313,0,630,633,0,345,135,211,83,0,108,5,0,0,633,49,173,46,27,0,50,886,156,87,1328,647,24,18,3438,52,19,34,448,1427,427,0,2544,761,1355,54607,30000,65184,32891,29292,3172,94,1854,3664,667,371,104,0,0,1966,260,630,226,72,867,185,835,1097,2031,195,1628,89

In [383]:
# recreate the list of columns from the 2000 dataset to match format 
columns = []
columns.append('tractid')
for column in df_MA_sample_2000.columns[1:]:
    column_without_year = column[:-2].lower()
    column_new_year = column_without_year +'12'
    columns.append(column_new_year)

In [400]:
# manually fix a couple that dont exactly match this format
columns[49] = 'hu12'
columns[51] = 'ohu12'
columns[21] = '12-Mar'

In [401]:
# subset df to match 2000 format
df_MA_sample_2010=df_MA_sample_2010[columns]

In [437]:
# merge so we can calculate % change

In [520]:
df_joined_sample = df_MA_sample_2000.merge(df_MA_sample_2010, on='tractid')

In [521]:
df_joined_sample.shape

(203, 133)

## Calculate change b/w censuses (sample count)

In [522]:
# this is where 2000 data starts 
i = 2
# this is where 2010 data starts
j = 67
# stop once you get to final column
while j < 134:
    try:
        df_joined_sample[f'{df_joined_sample.columns[i]}_POC'] = df_joined_sample.iloc[:,1:].apply(lambda x: 100*
                                    ((x[df_joined_sample.columns[j]]-x[df_joined_sample.columns[i]])
                                     /x[df_joined_sample.columns[i]]),axis=1)
    except ZeroDivisionError:
        'NaN'
        
    i +=1
    j +=1

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [524]:
df_joined_sample.to_csv('Sample_count_census_changes.csv')